In [21]:
from serpapi import GoogleSearch
import json
from newspaper import Article
import pandas as pd

import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [18]:


params = {
    "api_key": "666cf29c7b4a95e8fee44af175336637cb3d3c50d3b132a3dad308593d58c04f",           # https://serpapi.com/manage-api-key
    "engine": "google",         # serpapi parsing engine
    "q": "agriculture climate telangana",     # search query
    "gl": "in",                 # country from where search comes from
    "tbm": "nws"                # news results
    # other parameters such as language `hl` and number of news results `num`, etc.
}

search = GoogleSearch(params)   # where data extraction happens on the backend

pages = search.pagination()   # returns an iterator of all pages

titles = []
news_links = []
for page in pages:
    print(f"Current page: {page['serpapi_pagination']['current']}")

    for result in page["news_results"]:
        print(f"Title: {result['title']}\nLink: {result['link']}\n")
        titles.append(result['title'])
        news_links.append(result['link'])

https://serpapi.com/search
Current page: 1
Title: Budget allocations for agriculture must benefit small and marginal farmers
Link: https://www.bizzbuzz.news/opinion/budget-allocations-for-agriculture-must-benefit-small-and-marginal-farmers-1200446

Title: Telangana: Agriculture Minister attends ICRISAT’s conference
Link: https://www.siasat.com/telangana-agriculture-minister-attends-icrisats-conference-2531852/

Title: Bearing the brunt of climate change
Link: https://www.dailypioneer.com/2023/columnists/bearing-the-brunt-of-climate-change.html

Title: Impact of climate change on agriculture: Budget could have done more
Link: https://thefederal.com/business/budget-2023/impact-of-climate-change-on-agriculture-budget-could-have-done-more/

Title: Ivar Agro to Invest Rs 200 crore to set up a millet-based food ...
Link: https://agriculturepost.com/agribusiness/food-processing/ivar-agro-to-invest-rs-200-crore-to-set-up-a-state-of-the-art-millet-based-food-project-in-uttarakhand-2/

Title: Fa

In [ ]:
titles =[]
dates = []
authors =[]
texts = []
images =[]
movies =[]
keywords = []
summaries = []
links = []
for url in news_links:
    
    try:
        article = Article(url)
        article.download()

        article.parse()
        
        titles.append(article.title)
        dates.append(article.publish_date)
        authors.append(article.authors)
        texts.append(article.text)
        images.append(article.top_image)
        movies.append(article.movies)

        article.nlp()

        keywords.append(article.keywords)
        summaries.append(article.summary)

        links.append(url)
        print(article.title)

    except:
        pass

In [47]:
df = pd.DataFrame()

df['title'] = titles
df['date'] = dates
df['authors'] = authors
df['keywords'] = keywords
df['summary'] = summaries
df['text'] = texts
df['images'] = images
df['movies'] = movies
df['link'] = news_links

In [ ]:
df.to_csv('NEWS.csv', index =False)


df = pd.read_csv('NEWS.csv')

In [18]:
df_subset = df.reset_index()[['index', 'text']].copy()

In [21]:
df_subset['text'] = df_subset['text'].str.replace("[^a-zA-Z#]", " ")
#covert to lower-case
df_subset['text'] = df_subset['text'].str.casefold()
print (df_subset.head(10))

/tmp/ipykernel_18766/1394752374.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_subset['text'] = df_subset['text'].str.replace("[^a-zA-Z#]", " ")


   index                                               text
0      0  agriculture has been adversely impacted by cli...
1      1  hyderabad  telangana agriculture minister  s n...
2      2  women are the worst sufferers of climate chang...
3      3  what india needs is sustainable intensificatio...
4      4  with an initial investment of rs     crore  iv...
5      5  this is the eighth article in a    part series...
6      6  rewind  too taxing an idea  the proposal of do...
7      7  the agriculture budget lacks the overarching t...
8      8  india is today a      trillion economy  if the...
9      9  instalinks   help you think beyond the issue b...


In [22]:
# set up empty dataframe for staging output
df1=pd.DataFrame()
df1['index']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [ ]:
print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in df_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['index']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.index != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

In [ ]:

#merge dataframes
df_output = pd.merge(df.reset_index(), t_df_cleaned, on='index', how='inner')

In [32]:
df_output[["sentiment_score"]].describe()

,sentiment_score
count,275.000000
mean,0.588399
std,0.709480
min,-0.999500
25%,0.630900
50%,0.976000
75%,0.995200
max,1.000000
